In [1]:
import pandas as pd
import time
from datetime import datetime

In [2]:
data = pd.read_csv('data.tsv', sep='\t')

In [3]:
data.head()

,id,date,user_id,duration,medium,source,cost,order_id,amount_paid
0,40443,05.10.2016 23:18,1010,0.000926,seo,google,0.00,6243,20.2
1,35044,09.10.2016 21:40,1036,0.006493,sem,yandex,0.07,6145,15.6
2,40177,05.10.2016 3:23,1041,0.003380,email,promo,0.00,6128,13.2
3,39401,05.10.2016 23:19,1041,0.000463,sem,yandex,0.03,6697,9.8
4,41545,01.10.2016 4:57,1042,0.006493,sem,google,0.06,4510,14.8


In [4]:
def convert_to_datetime(row):
    return datetime.strptime(row['date'], '%d.%m.%Y %H:%M')

In [5]:
data['datetime'] = data.apply(convert_to_datetime, axis=1)

In [6]:
data.head()

,id,date,user_id,duration,medium,source,cost,order_id,amount_paid,datetime
0,40443,05.10.2016 23:18,1010,0.000926,seo,google,0.00,6243,20.2,2016-10-05 23:18:00
1,35044,09.10.2016 21:40,1036,0.006493,sem,yandex,0.07,6145,15.6,2016-10-09 21:40:00
2,40177,05.10.2016 3:23,1041,0.003380,email,promo,0.00,6128,13.2,2016-10-05 03:23:00
3,39401,05.10.2016 23:19,1041,0.000463,sem,yandex,0.03,6697,9.8,2016-10-05 23:19:00
4,41545,01.10.2016 4:57,1042,0.006493,sem,google,0.06,4510,14.8,2016-10-01 04:57:00


In [7]:
def make_unix_time(row):
    return time.mktime(row['datetime'].timetuple())

In [8]:
data['unixtime'] = data.apply(make_unix_time, axis=1)
data.head()

,id,date,user_id,duration,medium,source,cost,order_id,amount_paid,datetime,unixtime
0,40443,05.10.2016 23:18,1010,0.000926,seo,google,0.00,6243,20.2,2016-10-05 23:18:00,1.475699e+09
1,35044,09.10.2016 21:40,1036,0.006493,sem,yandex,0.07,6145,15.6,2016-10-09 21:40:00,1.476038e+09
2,40177,05.10.2016 3:23,1041,0.003380,email,promo,0.00,6128,13.2,2016-10-05 03:23:00,1.475627e+09
3,39401,05.10.2016 23:19,1041,0.000463,sem,yandex,0.03,6697,9.8,2016-10-05 23:19:00,1.475699e+09
4,41545,01.10.2016 4:57,1042,0.006493,sem,google,0.06,4510,14.8,2016-10-01 04:57:00,1.475287e+09


In [9]:
data_df = data.groupby('user_id').agg({'unixtime' : ['min', 'max']})

In [10]:
data_df['diff'] = data_df['unixtime']['max'] - data_df['unixtime']['min']

In [11]:
data_df.head()

unixtime                    diff
                  min           max          
user_id                                      
1010     1.475699e+09  1.475699e+09       0.0
1036     1.476038e+09  1.476038e+09       0.0
1041     1.475627e+09  1.475699e+09   71760.0
1042     1.475287e+09  1.475934e+09  647340.0
1047     1.475318e+09  1.475693e+09  374520.0

In [12]:
data_df_filtered = data_df[(data_df['diff'] !=0)]

In [13]:
data_df_filtered.head(20)

unixtime                    diff
                  min           max          
user_id                                      
1041     1.475627e+09  1.475699e+09   71760.0
1042     1.475287e+09  1.475934e+09  647340.0
1047     1.475318e+09  1.475693e+09  374520.0
1052     1.475824e+09  1.475945e+09  121020.0
1057     1.475569e+09  1.475970e+09  401700.0
1100     1.475321e+09  1.475929e+09  608340.0
1108     1.475285e+09  1.475991e+09  705780.0
1112     1.475634e+09  1.475961e+09  326280.0
1113     1.475738e+09  1.476004e+09  266400.0
1114     1.475680e+09  1.475766e+09   86280.0
1118     1.475353e+09  1.476037e+09  684060.0
1123     1.475674e+09  1.475721e+09   46620.0
1134     1.475546e+09  1.475744e+09  198060.0
1135     1.475327e+09  1.475957e+09  630720.0
1136     1.475350e+09  1.475525e+09  174900.0
1140     1.475471e+09  1.475564e+09   93300.0
1145     1.475627e+09  1.475716e+09   88140.0
1149     1.475385e+09  1.476041e+09  656040.0
1154     1.475399e+09  1.475611e+09  211680.0
1159     1.475562e+09  1.475577e+09   14760.0

In [14]:
result = data_df_filtered.groupby('user_id').mean()

In [15]:
result['days'] = result['diff'] // 86400

In [16]:
result.head()

unixtime                    diff days
                  min           max               
user_id                                           
1041     1.475627e+09  1.475699e+09   71760.0  0.0
1042     1.475287e+09  1.475934e+09  647340.0  7.0
1047     1.475318e+09  1.475693e+09  374520.0  4.0
1052     1.475824e+09  1.475945e+09  121020.0  1.0
1057     1.475569e+09  1.475970e+09  401700.0  4.0

In [17]:
result.drop(labels=['unixtime', 'diff'], axis=1).reset_index()

,user_id,days
,,
0,1041,0.0
1,1042,7.0
2,1047,4.0
3,1052,1.0
4,1057,4.0
5,1100,7.0
6,1108,8.0
7,1112,3.0
8,1113,3.0
